<a href="https://colab.research.google.com/github/MarkEbel/Image-retrievel/blob/main/ImageRetrievel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Orginial**

In [ ]:
from PIL import Image
import re
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, cuda
 
def buildHistogram(location):
  img = np.asarray(Image.open("./fruitnveg/"+ location))
  #img[row, column, channel (rgb)]
  values = []
  bins = 15
  for i in img:
    for j in i:
      tmp = ((((bins*j[0])/256)*bins)*bins) + (((bins*j[1])/256)*bins + (j[2]*(bins))/256)
      values.append(tmp)
  
  return values
  

#write ranked retrievel based on histograms and return top results
# use histogram intersection
def histogram_intersection(h1, h2):
    sm = 0;
    for i in range(15):
        sm += min(h1[i], h2[i])
    return sm

def search(file):
  v = buildHistogram(file)
  intersections = {}
  files = !ls fruitnveg
  filesSplitted = tokenize(str(files))
  filesSplitted.remove('[')
  filesSplitted.remove(']')

  for file in filesSplitted:
    intersections[file] = histogram_intersection(v, buildHistogram(file))

  sortedIntersections = {k: v for k, v in sorted(intersections.items(), key=lambda item: item[1], reverse = True)}
  return list(sortedIntersections.keys())[0:5]

DELIM = '[ \r\n\t;:,/\(\)\"\'-]+'
def tokenize(text):
    return re.split(DELIM, text.lower())

search("img0000.png")
 # find all histogram_intersections

**Attempt at GPU usage**

In [ ]:
from numba import cuda
from PIL import Image
import re
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, cuda
@jit(nopython=True)
def buildHistogram(location):
  img = np.asarray(Image.open("./fruitnveg/"+ location))
  #img[row, column, channel (rgb)]
  values = []
  bins = 15
  for i in img:
    for j in i:
      tmp = ((((bins*j[0])/256)*bins)*bins) + (((bins*j[1])/256)*bins + (j[2]*(bins))/256)
      values.append(tmp)
  
  return values
  

#write ranked retrievel based on histograms and return top results
# use histogram intersection
def histogram_intersection(h1, h2):
    sm = 0;
    for i in range(15):
        sm += min(h1[i], h2[i])
    return sm

DELIM = '[ \r\n\t;:,/\(\)\"\'-]+'
@jit(nopython=True)
def tokenize(text):
    return re.split(DELIM, text.lower())

@cuda.jit
def my_kernel(io_array):
    """
    Code for kernel.
    """
    # code here
    filesSplitted = tokenize(str(io_array))
    filesSplitted.remove('[')
    filesSplitted.remove(']')
    histograms = {}

    for file in filesSplitted:
      histograms[file] = buildHistogram(file)
    io_array = histograms

import numpy

# Create the data array - usually initialized some other way
data = numpy.ones(256)

# Set the number of threads in a block
threadsperblock = 32 

# Calculate the number of thread blocks in the grid
blockspergrid = (data.size + (threadsperblock - 1)) // threadsperblock

# Now start the kernel
data = !ls fruitnveg
my_kernel[blockspergrid, threadsperblock](data)

# Print the result
print(data)